In [ ]:
import ee
import pandas as pd
ee.Authenticate()

True

In [9]:
ee.Initialize()
file_path = 'filtered_storms_deleted_formatted.csv'
df = pd.read_csv(file_path)


In [29]:
import datetime
import geemap
def get_precipitation(lat, lon, time):
    # Convert time to datetime object
    time = datetime.datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    # Define the time window (1 hour before and after the provided time)
    start_time = ee.Date(time)  # start of the time window
    end_time = start_time.advance(1, 'hour')  # 1 hour after

    # Define the point for which we need precipitation data
    point = ee.Geometry.Point(lon, lat)

    # Get the GPM IMERG dataset for precipitation (version 7)
    dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_V07')
    precipitation = dataset.filterBounds(point) \
                           .filterDate(start_time, end_time) \
                           .select('precipitation') \
                           .mean()  # average over the time window
    
    # Reduce to the location (point) and get the precipitation value
    precip_value = precipitation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=1000  # resolution of the data in meters
    ).get('precipitation')

    return precip_value.getInfo()

precipitation_values = []

for index, row in df.iterrows():
    lat = row['usa_lat']
    lon = row['usa_lon']
    time = row['time']
    
    precip_value = get_precipitation(lat, lon, time)
    precipitation_values.append(precip_value)

df['precipitation'] = precipitation_values



In [ ]:
# Save the updated DataFrame to a new CSV file
output_file = 'data_usa.csv'
df.to_csv(output_file, index=False)

'data_usa.csv'

# Japan 
# "storms_JPN_formatted.csv" 

In [ ]:
file_path = 'storms_JPN_formatted.csv'
df = pd.read_csv(file_path)
# Apply the function to each row in the DataFrame
precipitation_values = []

for index, row in df.iterrows():
    lat = row['tokyo_lat']
    lon = row['tokyo_lon']
    time = row['time']
    
    precip_value = get_precipitation(lat, lon, time)
    precipitation_values.append(precip_value)

# Add the precipitation values as a new column to the DataFrame
df['precipitation'] = precipitation_values


In [13]:

df.to_csv('data_tokyo.csv', index=False)

In [14]:
file_path = 'storms_cma_formatted.csv'
df = pd.read_csv(file_path)
# Apply the function to each row in the DataFrame
precipitation_values = []

for index, row in df.iterrows():
    lat = row['cma_lat']
    lon = row['cma_lon']
    time = row['time']
    
    precip_value = get_precipitation(lat, lon, time)
    precipitation_values.append(precip_value)

# Add the precipitation values as a new column to the DataFrame
df['precipitation'] = precipitation_values

In [15]:

df.to_csv('data_cma.csv', index=False)